# Predict tags from interview scripts

## Initialize libraries

In [ ]:
# !pip install langchain_community
# !pip install langchain_aws

In [3]:
# !pip install faiss-gpu


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
# !pip install python-docx


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
# !pip install tqdm


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import pandas as pd
from botocore.config import Config
from langchain_community.vectorstores import FAISS
from langchain_aws import ChatBedrock, BedrockEmbeddings

In [7]:
from transcript_loader import load_transcripts
from tag_definitions import tags

In [8]:
from pprint import pprint

In [9]:
import transcript_tagger

In [10]:
import importlib

try:
    importlib.reload(transcript_tagger)
    print("Module successfully reloaded")
except Exception as e:
    print(f"Error reloading module: {e}")

Module successfully reloaded


# Preprocess the input data

## Loading the transcripts

In [11]:
folder_path = "data/transcripts_train"
train_transcripts = load_transcripts(folder_path)

train_participants = list(train_transcripts.keys())
train_participants

['P18 Jenny W',
 'P16 Sari N',
 'P2 Tegan K',
 'P17 Ruby V',
 'P14 Sandy R',
 'P12 Lindsay S',
 'P15 Ben W',
 'P13 Tina B',
 'P10 Ashantae O',
 'P11 Jamie M',
 'P1 Olga M']

In [25]:
folder_path = "data/transcripts_test_single"
test_transcripts = load_transcripts(folder_path)

test_participants = list(test_transcripts.keys())
test_participants

['P5 Joana J']

## Loading the manual tags

In [13]:
df = pd.read_csv("data/Raw Condens Data - highlights_export.csv")

In [14]:
df.head(2)

,Quote,Participants,Tag Group,Tag,Segment,Copy to Miro,Link
0,"[P3 Lia O] 'He's, you know, old school. So I u...",P3 Lia O,Caregiver,grocery,"Kids 0 - 10, Caregiver","[P3 Lia O] 'He's, you know, old school. So I u...",https://app.condens.io/session/XK6q3c/A6cYxXoK...
1,[P3 Lia O] 'He if he wants like a pizza or som...,P3 Lia O,Caregiver,food delivery,"Kids 0 - 10, Caregiver",[P3 Lia O] 'He if he wants like a pizza or som...,https://app.condens.io/session/XK6q3c/A6cYxXoK...


## Splitting Train and Test Transcripts

In [15]:
def contains_match(participant, all_names):
    participant_id = participant.split(' ')[0]
    return any(participant_id in name for name in all_names)

# Apply the matching function to filter rows

matching_mask = df['Participants'].apply(lambda x: contains_match(x, train_participants))


# Split the DataFrame into those that match and those that don't
df_train = df[matching_mask]
df_test = df[~matching_mask]

## Checking tag definitions

In [17]:
# # Iterate over all tags
# for tag in tags:
#     print(f"{tag.tag_group} - {tag.tag}")

## Creating Vector Store and Starting LLM Instance

In [26]:
bedrock_config = Config(
            retries={'max_attempts': 100},
            read_timeout=1000,
        )

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0",
    # model_id="cohere.embed-english-v3"
    config=bedrock_config,
)


llm_instance = ChatBedrock(
    # model_id= "anthropic.claude-3-haiku-20240307-v1:0",
    # model_id="anthropic.claude-3-sonnet-20240229-v1:0"
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    # model_id="anthropic.claude-3-5-sonnet-20241022-v2:0",
    # model_id="us.meta.llama3-2-90b-instruct-v1:0"
    # model_id="meta.llama3-1-405b-instruct-v1:0"
    # model_id="meta.llama3-1-70b-instruct-v1:0"
    region_name='us-east-1',
    config=bedrock_config,
    # model_kwargs={'temperature': 0.01} # Play around with this if your results isn't consistent
).with_retry()

Using the mapping to retrieve tag (not embedding it with the Quote)

In [27]:
quote_tag_map = df_train.set_index('Quote')['Tag'].to_dict()

In [28]:
next(iter(quote_tag_map.items()))

("[P11 Jamie] 'For one of your your grandparents. Yeah, Grandfather.'",
 'who i give care to')

In [29]:
all_quotes = list(quote_tag_map.keys())
all_tags = list(set(tag.lower() for tag in quote_tag_map.values()))

In [30]:
%%time
quote_vector_store = FAISS.from_texts(all_quotes, 
                                      embeddings,
                                      normalize_L2=True)

CPU times: user 2.63 s, sys: 104 ms, total: 2.73 s
Wall time: 1min 5s


In [31]:
%%time
tag_vector_store = FAISS.from_texts(all_tags,
                                    embeddings,
                                    normalize_L2=True)

CPU times: user 165 ms, sys: 7.14 ms, total: 172 ms
Wall time: 4.06 s


# Running the Tagger

In [32]:
import importlib

try:
    importlib.reload(transcript_tagger)
    print("Module successfully reloaded")
except Exception as e:
    print(f"Error reloading module: {e}")

Module successfully reloaded


In [33]:
tagger = transcript_tagger.TranscriptTagger(tags, llm_instance, quote_tag_map, quote_vector_store, tag_vector_store)

In [34]:
%%time
df_out = tagger.tag_transcripts(test_transcripts)

Tagging transcripts for each participant: 100%|██████████| 1/1 [05:08<00:00, 308.93s/it]

CPU times: user 12.3 s, sys: 177 ms, total: 12.4 s
Wall time: 5min 8s


In [102]:
# %%time
# df_out = tagger.tag_transcripts(test_transcripts)

Tagging transcripts for each participant: 100%|██████████| 7/7 [41:24<00:00, 354.94s/it]

CPU times: user 1min 21s, sys: 1.2 s, total: 1min 23s
Wall time: 41min 24s


In [36]:
df_out.head()

,Quote,Tag,Confidence,Tag Group,Participant
0,So I work as like a chief customer officer. So...,what i am responsible for,0.9,No Tag Group,P5 Joana J
1,I have 4 kids and one step kid.,kids,1.0,Family makeup,P5 Joana J
2,"Yeah, they all live in the home with me, so th...",kids,1.0,Family makeup,P5 Joana J
3,"OK, three 5, I think 7 and then 10.",kids,0.9,Family makeup,P5 Joana J
4,But most of the times traditionally I grew up ...,childcare,0.9,No Tag Group,P5 Joana J


In [35]:
df_out.to_csv('output_one_script.csv', index=False)